/home/zhibinz2/Documents/GitHub/Motor_cordination/1_over_f/data_analysis/hyperscanEEG_correlation/permutation20250515ramesh_code.ipynb

In [1]:
import numpy as np
from numpy import random
from matplotlib import pyplot as plt
from hdf5storage import loadmat,savemat
import pandas as pd

In [ ]:
def myimshow(data, title=None, cmap='bwr', vmin=-2, vmax=2):
    plt.figure(figsize=(8, 6), layout='tight')
    plt.imshow(data, cmap=cmap, vmin =vmin, vmax=vmax)
    plt.colorbar(location='bottom')
    plt.yticks(np.arange(data.shape[0]),labels=[r'$\delta$',r'$\theta$',r'$\alpha$',r'$\mu$',r'$\beta_1$',r'$\beta_2$',r'$\gamma$'])
    if title is not None:
        plt.title(title)
    plt.show()

In [ ]:
rng = random.default_rng(1234)

In [ ]:
outdict=loadmat('/home/zhibinz2/Documents/GitHub/Motor_coordination_code/cc3_syn.mat')
cc3_syn=outdict['cc3_syn'] # 3nx x 2 syn x 4 condi x36 tr x 7 freq x 32 chan
# 3nx means 3 different networkx measurements; 2syn means 2 task types (synch and synco); 4 condi means 4 conditions; 

In [ ]:
#degree centrality -dc efficiency -ef betweenness centrality -bc
#36 tr means 36 trials; 7 freq means 7 frequency bands; 32 chan means 32 channels
tasks = ['synch','synco']
feedback = ['ind','lead','follow','mutual']
dc = np.zeros((2*4*36,7,32))
ef = np.zeros((2*4*36,7,32))
bc = np.zeros((2*4*36,7,32))
t = np.array(np.zeros((2*4*36)),dtype = 'str')
f = np.array(np.zeros((2*4*36)),dtype = 'str')

for j in range(2): # 2 syn
    dc2 = np.zeros((4*36,7,32))
    ef2 = np.zeros((4*36,7,32))
    bc2 = np.zeros((4*36,7,32)) 
    f2 = np.array(np.zeros(4*36),dtype='str')
    for k in range(4):
        dc2[k*36:(k+1)*36,:,:]= cc3_syn[0,j,k,:,:,:]
        ef2[k*36:(k+1)*36,:,:]= cc3_syn[1,j,k,:,:,:]
        bc2[k*36:(k+1)*36,:,:]= cc3_syn[2,j,k,:,:,:]
        f2[k*36:(k+1)*36] = feedback[k] # label condition
    dc[j*4*36:(j+1)*4*36,:,:] = dc2
    ef[j*4*36:(j+1)*4*36,:,:] = ef2
    bc[j*4*36:(j+1)*4*36,:,:] = bc2
    f[j*4*36:(j+1)*4*36] = f2
    t[j*4*36:(j+1)*4*36] = tasks[j]


In [ ]:
# find the 144 indicies of following and mutual combined 
follow_mutual=(f == feedback[2]) | (f == feedback[3])
tr_ind=np.where(follow_mutual)
follow_mutual_ind=tr_ind[0]
follow_mutual_ind 

In [ ]:
#Lets consider instead the way we did it in the first draft of the paper where we creare a null distribution and z-score the data. 
nsims = 20000
x = rng.choice(follow_mutual_ind,size = (36,nsims),replace = True) # shuffle all 36 trials among only the 144 trials 20000 times


In [ ]:
# degree centrality
datax = np.zeros((nsims,7,32)) # the distribution of 20000 times
for j in range(nsims):
    datax[j,:,:] = np.mean(dc[x[:,j],:,:],axis  = 0) # average over 36 randomized trials
dxmean = np.mean(datax,axis=0) # average over 20000 sim
dxstd = np.std(datax,axis=0)

# zscore
# dataz = np.zeros((144,7,32)) # zscore (not needed, got updated to 7x 32 in each condition)

dcsynch = dict()
#Degree centrality 
for k in range(2,4):
    dataz= np.mean(dc[(t == 'synch') & ((f == feedback[k])),:,:],axis =0)
    dataz = (dataz - dxmean)/dxstd
    dcsynch[feedback[k]] = dataz

dcsynco = dict()
for k in range(2,4):
    dataz= np.mean(dc[(t == 'synco') & ((f == feedback[k])),:,:],axis =0)
    dataz = (dataz - dxmean)/dxstd
    dcsynco[feedback[k]] = dataz

dctask = dict()
for j in range(2):
    dataz= np.mean(dc[((t == tasks[j]) & follow_mutual )],axis =0)
    dataz = (dataz - dxmean)/dxstd
    dctask[tasks[j]] = dataz

dcfeed = dict()
for k in range(2,4):
    dataz= np.mean(dc[(f == feedback[k])],axis =0)
    dataz = (dataz - dxmean)/dxstd
    dcfeed[feedback[k]] = dataz


In [ ]:
savepath='/home/zhibinz2/Documents/GitHub/Motor_cordination/1_over_f/data_analysis/hyperscanEEG_correlation/zscore_20250515/'
savemat(savepath+'dcsynch.mat',  dcsynch)
savemat(savepath+'dcsynco.mat',  dcsynco)
savemat(savepath+'dctask.mat',   dctask)
savemat(savepath+'dcfeed.mat',   dcfeed)


In [ ]:

# betweenness centrality   
datax = np.zeros((nsims,7,32))
for j in range(nsims):
    datax[j,:,:] = np.mean(bc[x[:,j],:,:],axis  = 0)
dxmean = np.mean(datax,axis=0)
dxstd = np.std(datax,axis=0)

# dataz = np.zeros((288,7,32))

bcsynch = dict()
for k in range(2,4):
    dataz= np.mean(bc[(t == 'synch') & ((f == feedback[k])),:,:],axis =0)
    dataz = (dataz - dxmean)/dxstd
    bcsynch[feedback[k]] = dataz

bcsynco = dict()
for k in range(2,4):
    dataz= np.mean(bc[(t == 'synco') & ((f == feedback[k])),:,:],axis =0)
    dataz = (dataz - dxmean)/dxstd
    bcsynco[feedback[k]] = dataz

bctask = dict()
for j in range(2):
    dataz= np.mean(bc[(t == tasks[j]) & follow_mutual],axis =0)
    dataz = (dataz - dxmean)/dxstd
    bctask[tasks[j]] = dataz

bcfeed = dict()
for k in range(2,4):
    dataz= np.mean(bc[(f == feedback[k])],axis =0)
    dataz = (dataz - dxmean)/dxstd
    bcfeed[feedback[k]] = dataz
         

In [ ]:
savepath='/home/zhibinz2/Documents/GitHub/Motor_cordination/1_over_f/data_analysis/hyperscanEEG_correlation/zscore_20250515/'
savemat(savepath+'bcsynch.mat',  bcsynch)
savemat(savepath+'bcsynco.mat',  bcsynco)
savemat(savepath+'bctask.mat',   bctask)
savemat(savepath+'bcfeed.mat',   bcfeed)


plot

In [ ]:
for k in range(2,4):
    myimshow(dcsynch[feedback[k]], title  = 'DC synch ' + feedback[k])
    print(np.where(np.abs(dcsynch[feedback[k]]) > 1.96))

# savemat('dcsynch.mat', {'dcsynch': dcsynch})

In [ ]:
for k in range(2,4):
    myimshow(dcsynco[feedback[k]],title = 'DC synco ' + feedback[k])
    print(np.where(np.abs(dcsynco[feedback[k]]) > 1.96))

# savemat('dcsynco.mat', {'dcsynco': dcsynco})

In [ ]:
for k in range(2,4):
    myimshow(bcsynch[feedback[k]], title  = 'BC synch ' + feedback[k])
    print(np.where(np.abs(bcsynch[feedback[k]]) > 1.96))

# savemat('bcsynch.mat', {'bcsynch': bcsynch})

In [ ]:
for k in range(2,4):
    myimshow(bcsynco[feedback[k]], title  = 'BC synco ' + feedback[k])
    print(np.where(np.abs(bcsynco[feedback[k]]) > 1.96))

# savemat('bcsynco.mat', {'bcsynco': bcsynco})